In [5]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np
from scipy.sparse import dok_matrix, coo_matrix, csr_matrix, lil_matrix
import scipy


import settings

In [2]:
vk_func_getMembIDList = VkFunction(args=('x', 'y'), code='''
    var offset = %(y)s;
    var IDList = [];
    var i = 0;
    while (i < 25)
    {
        IDList.push(API.groups.getMembers({"group_id": %(x)s, "offset" : offset + i * 1000})["items"]);
        i = i + 1;
    }
    return IDList;
''')

In [3]:
MAX_COMM_ID = 172741353
MAX_USER_ID = 550000000

In [5]:
IDMatr = dok_matrix((MAX_COMM_ID, MAX_USER_ID), dtype=np.int8)

In [22]:
IDMatr[(2, 235)]

0

In [6]:
File = shelve.open(f"alive_dict_from70000000_to79525017")
MQDict = File['info']
File.close()

In [8]:
users = 0
n = 0

for _, user in MQDict.items():
    users += user


In [11]:
len(MQDict)

7601

In [13]:
498780428 // 7601

65620

In [7]:
%%time

File = shelve.open(f"alive_dict_from70000000_to79525017")
MQDict = File['info']
File.close()

vk_session = vk_api.VkApi(token=settings.MY_COMM_TOKEN) 
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
    

vk = vk_session.get_api()

for start_id in range(76000000, 79500000, 100000):
    stop_id = start_id + 100000

    IDMatr = dok_matrix((MAX_COMM_ID, MAX_USER_ID), dtype=np.int8)
    
    for group_id, quan in MQDict.items():
        if start_id <= group_id <= stop_id:
            IDlist = []
            for offset in range(0, quan, 25000):
                IDlist += vk_func_getMembIDList(vk, x=group_id, y=offset)
        
        
            for user_id in IDlist:
                if user_id is not None:
                    IDMatr[(group_id, user_id)] = 1 
            
    scipy.sparse.save_npz(f"./data/user_ID_from_group{start_id}_to_group{stop_id}.npz", IDMatr.tocsr())

Login is required to auth
CPU times: user 10min 9s, sys: 34.8 s, total: 10min 44s
Wall time: 1h 27min 44s


In [23]:
IDMatr

<172741353x550000000 sparse matrix of type '<class 'numpy.int8'>'
	with 5001241 stored elements in Dictionary Of Keys format>

In [ ]:
File = shelve.open(f"user_ID_from_group{start_id}_to_group{stop_id}")
File['info'] = IDMatr
File.close()

In [11]:
import sys

In [24]:
IDMatr.tocsr()

<172741353x550000000 sparse matrix of type '<class 'numpy.int8'>'
	with 0 stored elements in Compressed Sparse Row format>

In [12]:
sys.getsizeof(IDMatr.tocsr()) // 1000 // 1000

0

In [8]:
import scipy.sparse

In [24]:
scipy.sparse.save_npz(f"user_ID_from_group{start_id}_to_group{stop_id}.npz", IDMatr.tocsr())

KeyboardInterrupt: 